In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122164")
exp = Experiment(workspace=ws, name="quick-starts-ws-122164")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FRZ7YPADW to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-122164
Azure region: southcentralus
Subscription id: 572f8abf-a1a0-4b78-8c6d-3630739c72b5
Resource group: aml-quickstarts-122164


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cluster-notebook"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.1, 0.3, 0.5, 0.7, 1.0),
    '--max_iter': choice(100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

best_model = hdr.get_best_run_by_primary_metric()
best_model

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-122164,HD_7cafa47b-70f2-4afe-a042-31d9bfa4fa73_3,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
best_model.download_file("outputs/Hyperdrive_Model.joblib", "Best_Model.joblib")

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path)

In [8]:
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

training_data = pd.concat([x_train, y_train], axis=1)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
30921,33,1,0,0,0,5,1,145,4,999,...,1,0,0,0,1,0,0,0,0,0
6960,31,1,0,1,0,5,2,239,4,999,...,0,0,0,0,0,0,1,0,0,0
31334,39,0,0,1,0,7,2,132,6,999,...,0,0,0,0,1,0,0,0,0,0
4569,34,0,0,0,0,6,4,432,8,999,...,1,0,0,0,0,0,1,0,0,0
8795,30,1,0,0,0,5,3,129,1,999,...,0,0,0,1,0,0,0,0,0,0


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_data,
    label_column_name='y',
    n_cross_validations=5
)

In [10]:
# Submit your automl run

remote_run = exp.submit(config=automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_0bb8449e-6f3b-40c5-aeb2-9a7cb131c2ef

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [15]:
remote_run.get_best_child()

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-122164,AutoML_0bb8449e-6f3b-40c5-aeb2-9a7cb131c2ef_58,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
remote_run.get_output()

(Run(Experiment: quick-starts-ws-122164,
 Id: AutoML_0bb8449e-6f3b-40c5-aeb2-9a7cb131c2ef_58,
 Type: None,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                   l1_ratio=0.3877551020408163,
                                                                                                   learning_rate='constant',
 

In [16]:
_, automl_best_model = remote_run.get_output()
automl_best_model.steps[-1]

('prefittedsoftvotingclassifier',
 PreFittedSoftVotingClassifier(classification_labels=None,
                               estimators=[('46',
                                            Pipeline(memory=None,
                                                     steps=[('standardscalerwrapper',
                                                             <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f56a512f208>),
                                                            ('xgboostclassifier',
                                                             XGBoostClassifier(base_score=0.5,
                                                                               booster='gbtree',
                                                                               colsample_bylevel=1,
                                                                               colsample_bynode=1,
                                                                           

In [17]:
# Retrieve and save your best automl model.

joblib.dump(automl_best_model, "AutoML_Model.pkl")

['AutoML_Model.pkl']

In [18]:
# Clean Up Compute Cluster

AmlCompute.delete(cpu_cluster)

Current provisioning state of AmlCompute is "Deleting"

